## [seq2seq](https://keras.io/examples/nlp/lstm_seq2seq/)
- 起因:
    - 從line engineering學到這是它們embedding的其中一個, 我也剛好都沒有機會去實作
    - 這是toy example的機器翻譯版本

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
!!curl -O http://www.manythings.org/anki/fra-eng.zip
!!unzip fra-eng.zip

['Archive:  fra-eng.zip',
 '  inflating: _about.txt              ',
 '  inflating: fra.txt                 ']

#### Configuation

In [3]:
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000
data_path = 'fra.txt'

#### 資料準備

In [7]:
# 向量化資料

input_texts = []
target_texts = []
input_characters = set()            # 用來~
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[:min(num_samples, len(lines)-1)]:
    input_text, target_text, _ = line.split('\t')       # 可以觀察fra.txt檔案, 後面的檔案不是我們要的!
    # 參考範例使用 "\t" 當作答案的開頭, "\n" 當作結束, 我使用"beg", "ens", 好像不行, 因為這是character level
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

# 排序以及token大小
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

# token mapping ---> char:token index
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([char, i] for i, char in enumerate(target_characters))

# 要做one-hot
encoder_input_data = np.zeros(
    shape=(len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32'
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
# 將對應出現位置放上1, one-hot的概念
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0    # ?
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
        decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [15]:
print(encoder_input_data.shape)     # (num_samples, seq_len, embedding_size(one-hot))
encoder_input_data[0]

(10000, 15, 71)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [16]:
print(decoder_input_data.shape)         # (num_samples, seq_len, embedding_size)
print(decoder_input_data[0])

(10000, 59, 93)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]


#### 建立model
- 這裡應該可以包裝成一個Model, 但好像也沒需要...前面那些encoder可以在init那邊發生

In [17]:
# 定義輸入序列(原始)並處理
encoder_inputs = tf.keras.Input(shape=(None, num_encoder_tokens))
encoder = tf.keras.layers.LSTM(latent_dim, return_state=True)       # 會return state
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# encoder_outputs是我們不需要的, 我們只需要keep states
encoder_states = [state_h, state_c]

# 設定decoder, 使用 `encoder_states` 當作初始狀態.
decoder_inputs = tf.keras.Input(shape=(None, num_decoder_tokens))

# 設定我們的decoder return full output sequences
# 並且return 內部 states, 我們不使用return states in the training model, 但是會使用在推論階段
# 因為訓練時可以teaching force!!
decoder_lstm = tf.keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# 定義模型
model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

#### 訓練模型

In [18]:
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['acc']
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)
model.save('seq2seq_model.h5')

Epoch 1/100
125/125 [==============================] - 8s 41ms/step - loss: 1.8119 - acc: 0.7036 - val_loss: 1.0776 - val_acc: 0.7055
Epoch 2/100
125/125 [==============================] - 4s 32ms/step - loss: 0.9769 - acc: 0.7441 - val_loss: 0.9888 - val_acc: 0.7138
Epoch 3/100
125/125 [==============================] - 4s 32ms/step - loss: 0.8305 - acc: 0.7731 - val_loss: 0.8549 - val_acc: 0.7704
Epoch 4/100
125/125 [==============================] - 4s 32ms/step - loss: 0.7226 - acc: 0.8024 - val_loss: 0.7643 - val_acc: 0.7790
Epoch 5/100
125/125 [==============================] - 4s 31ms/step - loss: 0.6427 - acc: 0.8159 - val_loss: 0.7025 - val_acc: 0.7959
Epoch 6/100
125/125 [==============================] - 4s 32ms/step - loss: 0.5914 - acc: 0.8300 - val_loss: 0.6657 - val_acc: 0.8068
Epoch 7/100
125/125 [==============================] - 4s 32ms/step - loss: 0.5630 - acc: 0.8354 - val_loss: 0.6377 - val_acc: 0.8127
Epoch 8/100
125/125 [==============================] - 4s 32ms

In [19]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, None, 71)]   0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, None, 93)]   0                                            
__________________________________________________________________________________________________
lstm_4 (LSTM)                   [(None, 256), (None, 335872      input_5[0][0]                    
__________________________________________________________________________________________________
lstm_5 (LSTM)                   [(None, None, 256),  358400      input_6[0][0]                    
                                                                 lstm_4[0][1]               

#### 推論、預測(沒有teaching force了)!
1. encoder 輸入並且取回decoder state
2. 執行一次decoder with initial state and "start of seq"符號. 達到輸出即為下一次的target token(因為沒辦法teaching force)
3. 之後就有current state, current target token可以繼續

In [21]:
# 模型讀取
model = tf.keras.models.load_model('seq2seq_model.h5')

encoder_inputs = model.input[0]
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = tf.keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = tf.keras.Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = tf.keras.Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = tf.keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# 將輸入lookup回去人看得懂的語言
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # encoder 輸入得到state vectors
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [22]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Marche.

-
Input sentence: Go.
Decoded sentence: Marche.

-
Input sentence: Go.
Decoded sentence: Marche.

-
Input sentence: Hi.
Decoded sentence: Salut.

-
Input sentence: Hi.
Decoded sentence: Salut.

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run!
Decoded sentence: File !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

-
Input sentence: Run.
Decoded sentence: Fuyez !

